In [ ]:
pip install tensorflow gensim scikit-learn

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer


In [ ]:
categories = ['rec.sport.baseball', 'sci.space', 'comp.graphics', 'talk.politics.mideast']
newsgroups = fetch_20newsgroups(subset='all', categories=categories)
texts = newsgroups.data
labels = newsgroups.target

In [ ]:
le=LabelEncoder()
y=le.fit_transform(labels)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)

In [ ]:
max_length = 200
X = pad_sequences(X, maxlen=max_length)

In [ ]:
sentences = [text.split() for text in texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length,
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(categories), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(X, y, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 22s 333ms/step - accuracy: 0.2980 - loss: 1.3727 - val_accuracy: 0.4403 - val_loss: 1.2454
Epoch 2/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 312ms/step - accuracy: 0.4394 - loss: 1.2364 - val_accuracy: 0.4788 - val_loss: 1.1676
Epoch 3/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 314ms/step - accuracy: 0.5183 - loss: 1.1150 - val_accuracy: 0.5610 - val_loss: 1.0307
Epoch 4/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 15s 305ms/step - accuracy: 0.5588 - loss: 1.0474 - val_accuracy: 0.5956 - val_loss: 0.9678
Epoch 5/5
49/49 ━━━━━━━━━━━━━━━━━━━━ 21s 316ms/step - accuracy: 0.5788 - loss: 0.9830 - val_accuracy: 0.6175 - val_loss: 0.9066


In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 100)       │     4,990,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        42,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,117,902 (19.52 MB)

 Trainable params: 42,500 (166.02 KB)

 Non-trainable params: 4,990,400 (19.04 MB)

 Optimizer params: 85,002 (332.04 KB)

In [ ]:
loss, accuracy = model.evaluate(X, y)
print("Loss:", loss)
print("Accuracy:", accuracy)

122/122 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.6593 - loss: 0.8252
Loss: 0.8480328917503357
Accuracy: 0.6535695791244507
